- Exercise 1: Connect to your database server and print its version
- Exercise 2: Fetch Hospital and Doctor Information using hospital Id and doctor Id
- Exercise 3: Get the list Of doctors as per the given specialty and salary
- Exercise 4: Get a list of doctors from a given hospital
- Exercise 5: Update doctor experience in years
- Exercise 6: Close the connection and verify it.

In [2]:
# Importing libraries
import sqlite3
import pandas as pd

#### Question 1: Connect to your database server and print its version
--- 

In [48]:
# Question 1: Connect to your database server and print its version
try:
  # Create a connection object
  sqlconnection = sqlite3.connect("HospitalInfo.db")
  
  # Print the SQLite version
  print("Question 1:\n")
  print("Database connected successfully\n")
  version_query = "SELECT sqlite_version();"
  version = pd.read_sql_query(version_query, sqlconnection)
  print("SQLite version:", version.iloc[0, 0])

except sqlite3.Error as error:
  print(f"Error occurred: {error}")

Question 1:

Database connected successfully

SQLite version: 3.43.2


In [10]:
# Let us find the table names and columns information for each table
try:
    # Create a connection object
    sqlconnection = sqlite3.connect("HospitalInfo.db")
    cursor = sqlconnection.cursor()

    # Get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    print("\nTables in the database:")
    for table in tables:
        table_name = table[0]
        print(f"\nTable: {table_name}")
        
        # Get column information for each table
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        
        # Create a DataFrame to display the information nicely
        df = pd.DataFrame(columns, columns=['cid', 'name', 'type', 'notnull', 'dflt_value', 'pk'])
        print(df[['name', 'type']])  # Display only column name and type

except sqlite3.Error as error:
    print(f"Error occurred: {error}")


Tables in the database:

Table: Hospital
       name     type
0        ID  INTEGER
1      Name     TEXT
2  BedCount  INTEGER

Table: Doctor
          name     type
0        DocID  INTEGER
1      DocName     TEXT
2   HospitalID  INTEGER
3  JoiningDate     BLOB
4   Speciality     TEXT
5          Exp     TEXT
6       salary     REAL


#### Question 2 : Fetching Hospital and Doctor information using Hospital ID and Doctor ID
---

In [23]:
# Fetching Hospital and Doctor information using Hospital ID and Doctor ID
try:
    hospital_id = int(input("Enter Hospital ID: "))
    query_hospital = "SELECT * FROM Hospital WHERE ID = ?"
    doctor_id = int(input("Enter Doctor ID: "))
    query_doctor = "SELECT * FROM Doctor WHERE DocID = ?"

    
    # Execute the query and fetch results
    df_hospital = pd.read_sql_query(query_hospital, sqlconnection, params=(hospital_id,))
    df_doctor = pd.read_sql_query(query_doctor, sqlconnection, params=(doctor_id,))
    
    # Print the result
    if not df_hospital.empty:
        print(f"\nFetching Hospital information for ID: {hospital_id}")
        print(df_hospital)
    else:
        print(f"No matching records found for Hospital with ID {hospital_id}.")

    if not df_doctor.empty:
        print(f"\nFetching Doctor information for ID: {doctor_id}")
        print(df_doctor)
    else:
        print(f"No matching records found for Doctor with ID {doctor_id}.")    

except ValueError:
    print("Please enter valid integer IDs.")
except pd.io.sql.DatabaseError as error:
    print(f"Error occurred: {error}")


Fetching Hospital information for ID: 104
    ID             Name  BedCount
0  104  Clevland Clinic        30

Fetching Doctor information for ID: 1
   DocID  DocName  HospitalID JoiningDate Speciality Exp    salary
0      1  Michael         101  2005-02-10  Pediatric  10  900000.0


#### Question 3 : Get the list Of doctors as per the given speciality and salary
---

In [25]:
# Get the list Of doctors as per the given speciality and salary
try:
    speciality = input("Enter Speciality information: ").lower()
    salary = float(input("Enter Salary information: "))
    query_doctor = "SELECT * FROM Doctor WHERE LOWER(Speciality) = ? and salary = ?"

    
    # Execute the query and fetch results
    df_doctor = pd.read_sql_query(query_doctor, sqlconnection, params=(speciality,salary))
    
    if not df_doctor.empty:
        print(f"\nFetching Doctor information for speciality `{speciality}` and salary `{salary}`\n")
        print(df_doctor)
    else:
        print(f"\nNo matching records found for Doctor with speciality `{speciality}` and salary `{salary}`.\n")    

except ValueError:
    print("Please enter valid format for speciality and salary.")
except pd.io.sql.DatabaseError as error:
    print(f"Error occurred: {error}")


Fetching Doctor information for speciality `pediatric` and salary `900000.0`

   DocID  DocName  HospitalID JoiningDate Speciality Exp    salary
0      1  Michael         101  2005-02-10  Pediatric  10  900000.0


#### Exercise 4: Get a list of doctors from a given hospital
---

In [37]:
# Get the list Of doctors from a given hospital
try:
    hospital_id = int(input("Enter Hospital ID: "))
    query = """
    SELECT D.*
    FROM Doctor D
    JOIN Hospital H ON D.HospitalID = H.ID
    WHERE H.ID = ?
    """

    # Execute the query and fetch results
    df = pd.read_sql_query(query, sqlconnection, params=(hospital_id, ))
    
    if not df.empty:
        print(f"\nFetching Doctor information for hospital with id `{hospital_id}`\n")
        print(df)
    else:
        print(f"\nNo matching records found for Doctor.\n")    

except ValueError:
    print("Please enter valid format for ID.")
except pd.io.sql.DatabaseError as error:
    print(f"Error occurred: {error}")


Fetching Doctor information for hospital with id `104`

   DocID DocName  HospitalID JoiningDate Speciality Exp    salary
0      6  Robert         104  1998-09-04      Gyane  14  940000.0


#### Exercise 5: Update doctor experience in years
---

In [44]:
try:
    doctor_id = int(input("Enter Doctor ID: "))
    new_experience = input("Enter new experience (in years): ")

    # Check if the doctor exists
    find_doctor = "SELECT * FROM Doctor WHERE DocID = ?"
    cursor.execute(find_doctor, (doctor_id,))
    result = cursor.fetchone()

    # Update the doctor's experience if the doctor exists
    if result:
        print(result)
        update_query = "UPDATE Doctor SET Exp = ? WHERE DocID = ?"
        cursor.execute(update_query, (new_experience, doctor_id))
        sqlconnection.commit()
        print(f"\nExperience updated for Doctor ID {doctor_id}")
        cursor.execute(find_doctor, (doctor_id,))
        print(cursor.fetchone())
    else:
        print(f"No doctor found with ID {doctor_id}") 

except ValueError:
    print("Please enter valid format for ID.")
except pd.io.sql.DatabaseError as error:
    print(f"Error occurred: {error}")

(5, 'Karen', 103, '2020-09-05', 'Oncologist', '4', 1100000.0)

Experience updated for Doctor ID 5
(5, 'Karen', 103, '2020-09-05', 'Oncologist', '10', 1100000.0)


#### Exercise 6: Close the connection and verify it.
---

In [50]:
# Closing the connection
if 'sqlconnection' in locals():
      sqlconnection.close()
      print("\nDatabase connection closed.")


Database connection closed.


In [51]:
# Verify if the connection is closed
try:
    sqlconnection.execute("SELECT 1")
    print("The connection is still open.")
except:
    print("The connection is closed.")

The connection is closed.
